In [23]:
import boto3
from tqdm import tqdm

First I had to use [this repo](https://github.jpl.nasa.gov/cloud/Access-Key-Generation) to gain an AWS key pair. With this, I can download the large files from the bucket

In [32]:
bucket = 'aria-dev-isl-fwd-torresal'
key_0 = 'ALOS2_SLC/WD1/20150301/ALOS2041643200-150301.zip'
key_1 = 'ALOS2_SLC/WD1/20150412/ALOS2047853200-150412.zip'

local_path_0 = key_0.split('/')[-1]
local_path_1 = key_1.split('/')[-1]

session = boto3.session.Session(profile_name='saml-pub')
s3 = boto3.resource('s3')

source: https://stackoverflow.com/questions/56566785/how-to-use-boto3-to-retrieve-s3-file-size

In [33]:
client = session.client('s3')
response = client.head_object(Bucket=bucket,
                              Key=key_0)

total_bytes_0 = response['ContentLength']
total_bytes_0 * 9.31e-10

55.691018745383

In [34]:
client = session.client('s3')
response = client.head_object(Bucket=bucket,
                              Key=key_1)

total_bytes_1 = response['ContentLength']
total_bytes_1 * 9.31e-10

55.691018608526

Source: https://github.com/tqdm/tqdm/issues/89

In [35]:
def my_hook(t):
    """
    Wraps tqdm instance. Don't forget to close() or __exit__()
    the tqdm instance once you're done with it (easiest using `with` syntax).
    """
    last_b = [0]

    def inner(b=1, tsize=None, bsize=1):
        """
        b  : int, optional
            Number of blocks just transferred [default: 1].
        tsize  : int, optional
            Total size (in tqdm units). If [default: None] remains unchanged.
        bsize  : int, optional
            Size of each block (in tqdm units) [default: 1].
        """
        if tsize is not None:
            t.total = tsize
        t.update((b - last_b[0]) * bsize)
        last_b[0] = b
    return inner

In [39]:
with tqdm(total=total_bytes_0, unit='B', unit_scale=True, desc=FILE_NAME) as t:
    client.download_fileobj(bucket, key_0, open(local_path_0, 'wb'), Callback=hook(t))

ALOS2041643200-150301.zip: 100%|██████████| 59.8G/59.8G [05:05<00:00, 196MB/s]    


In [40]:
with tqdm(total=total_bytes_1, unit='B', unit_scale=True, desc=FILE_NAME) as t:
    client.download_fileobj(bucket, key_1, open(local_path_1, 'wb'), Callback=hook(t))

ALOS2041643200-150301.zip: 100%|██████████| 59.8G/59.8G [06:46<00:00, 147MB/s]    
